In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch


import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


# Example:

### Write out a trimmed-down forcefield file that only has the parameters we're using.

#### This works with our typical workflow:

1. Create molecule or system in mBuild
2. Use an existing focefield along with Foyer to apply the forcefield
3. Write out a useable, trimmed down foyer forcefield file

In [ ]:
# mBuild system
alkane = mb.load("CCCCC", smiles=True)
alkane_box = mb.fill_box(alkane, n_compounds=10, box=[2, 2, 2])

# opls forcefield form foyer
#ff = foyer.forcefields.get_forcefield(name="opls")
opls = foyer.Forcefield(name="oplsaa")

# parmed structure
alkane_pmd = opls.apply(alkane_box)

In [ ]:
# Save the trimmed xml file:
parmed_to_foyer_xml(
    structure=alkane_pmd,
    ff=opls,
    file_name="alkane_opls.xml",
)

In [ ]:
# Test out the xml file we created; we should be able to use it directly with Foyer:
ff_test = foyer.Forcefield(forcefield_files="alkane_opls.xml")
alkane_pmd_test = ff_test.apply(alkane_box)

### Try it out with GAFF

In [ ]:
gaff = foyer.Forcefield(forcefield_files="/Users/madilyn/Projects/repos/GAFF-foyer/gafffoyer/xml/gaff.xml")
alkane_pmd = gaff.apply(alkane_box)

parmed_to_foyer_xml(
    structure=alkane_pmd,
    ff=gaff,
    file_name="alkane_gaff.xml",
    torsion_type="periodic"
)

gaff_test = foyer.Forcefield(forcefield_files="alkane_gaff.xml")
alkane_gaff_test = gaff_test.apply(alkane_box)

# Example:

### Create a foyer xml file we can use with a abstract system

#### Workflow:
1. Create molecule or system in mBuild
2. Create data structure containing our abstract forcefield parameters
3. Write out a useable Foyer forcefield file

In [ ]:
# Example: Creating a bead-spring model of a polymer
mer = mb.Compound()
mer.add(mb.Compound(name="A", pos=[0,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[0.5,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[1.0,0,0], mass=1))
mer.freud_generate_bonds(name_a="A", name_b="A", dmax=0.51, dmin=0.049)

chain = Polymer()
chain.add_monomer(mer, indices=[0, 2], replace=False, orientation=[[-1, 0, 0], [1,0,0]], separation=0.5)
chain.build(n=3, add_hydrogens=False)
chain.visualize().show()

box_of_chains = mb.fill_box(compound=chain, n_compounds=10, box=[10, 10, 10])
box_of_chains.visualize().show()

In [ ]:
bond_parameters = {
    ("A", "A"): {"k": 50, "l0": 0.5}
}

angle_parameters = {
    ("A", "A", "A"): {"k": 5, "t0": 2.2}
}

dihedral_parameters = {
    ("A", "A", "A", "A"): {
        "periodicity": [2],
        "k": [10],
        "phase": [3.14]
    }
}

non_bonded_parameters = {
    "A": {"sigma": 1.0, "epsilon": 1.0, "charge": 0}
}

In [ ]:
mbuild_to_foyer_xml(
    file_name="testmb.xml",
    compound=box_of_chains,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    non_bonded_params=non_bonded_parameters,
)

In [ ]:
ff_test = foyer.Forcefield(forcefield_files="testmb.xml")
box_of_chains_typed = ff_test.apply(box_of_chains)

In [ ]:
snap, hoomd_ff, refs = create_hoomd_forcefield(structure=box_of_chains_typed, auto_scale=False, r_cut=25)

In [ ]:
print(snap.angles.types)
print(snap.dihedrals.types)

In [ ]:
print("Pairs:", hoomd_ff[0].params[('A', 'A')])
print("Bonds:", hoomd_ff[3].params["A-A"])
print("Angles:", hoomd_ff[4].params["A-A-A"])
print("Dihedrals:", hoomd_ff[5].params["A-A-A-A"])

# Example:

### Fitting espaloma into our MoSDeF-Hoomd Workflow:

In [2]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
benzene = mb.load(smiles, smiles=True)
benzene.visualize().show()

molecule = Molecule.from_smiles(smiles)
molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Find the set of particle type parameters:

In [4]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types:
        particle_types.append((charge, sigma, epsilon))
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon))

### `particle_types`  


#### This is the set (i.e. no duplicates) of the particle types found by espaloma

In [7]:
print(particle_types)

[(Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)), (Quantity(value=0.13, unit=elementary charge), Quantity(value=0.25996424595335105, unit=nanometer), Quantity(value=0.06276, unit=kilojoule/mole))]


In [ ]:
print(len(particle_types))

In [ ]:
for p in particle_types:
    print(p)
    print()

### `particle_type_dict`  

#### This is a dictionary that points to the parameters for each particle in benzene

#### `key: value`
#### `mbuild particle index: particle types index`

#### We have 12 total particles, but only 2 particle types

In [ ]:
len(particle_type_dict)

In [ ]:
print(particle_type_dict)

In [9]:
for p in particle_type_dict:
    print(particle_type_dict[p])

0
0
0
0
0
0
1
1
1
1
1
1


In [13]:
start = 0
for idx, type_num in enumerate(particle_type_dict):
    if particle_type_dict[type_num] == last:  
        

IndentationError: expected an indented block (1615376094.py, line 3)